In [1]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv('watermelon2.csv')

In [7]:
feature=dataset.columns.tolist()[1:]

In [8]:
data

,编码,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖量,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是
5,6,青绿,稍蜷,浊响,清晰,稍凹,软粘,0.403,0.237,是
6,7,乌黑,稍蜷,浊响,稍糊,稍凹,软粘,0.481,0.149,是
7,8,乌黑,稍蜷,浊响,清晰,稍凹,硬滑,0.437,0.211,是
8,9,乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,0.666,0.091,否
9,10,青绿,硬挺,清脆,清晰,平坦,软粘,0.243,0.267,否


In [9]:
data = np.array(data)
data = np.delete(data,0,1)#删除第一列编码 

### 信息熵
$$H(D)=- \sum_{i=1}^{k}(p_i\times logp_i)$$
$p_i$表示第i类的概率

In [10]:
from math import log
from collections import Counter

In [11]:
def calShannonEnt(dataset):
    numEntries = len(dataset)
    labelsCount = Counter(dataset[:,-1])
    shannonEnt = 0.0
    for key in labelsCount:
        prob = labelsCount[key]/numEntries
        shannonEnt -=prob*log(prob,2)
    return shannonEnt

In [12]:
calShannonEnt(data)

0.9975025463691153

### 信息增益
$$G(D,A)=H(D)-H(D|A)$$
$$H(D|A)=-\sum_{i=1}^{k}p_{i}H(D|A=a_j)H$$

首先需要筛选出特定特征以及特定特征取值的数据集 $D|A=a_j$，再根据公式计算条件熵。

In [13]:
#离散数据的分离
def splitDataSet(dataSet,axis,value):
    """
    需要计算H(D|A)，则需要把计算该特征的数据分离出来，这里适用于离散变量
    :param dataset:数据集 array
    :param axis: 特征，第几列
    :param value: 特征所取的值
    :return:
    """
    numberset = np.where(dataSet[:,axis] == value)
    return dataSet[numberset,][0]

In [14]:
#计算离散数据的信息增益
def calconEnt(dataset,axis):
    feature = Counter(dataset[:,axis])
    LEN = len(dataset[:,axis])
    conEnt = 0.0
    for key in feature:
        spdata = splitDataSet(dataset,axis,key)
        conEnt += float(feature[key]/LEN*calShannonEnt(spdata))
    return conEnt

In [15]:
#调用 第三列数据中取值为 清晰 的数据
splitDataSet(data,3,'清晰')

array([['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.6970000000000001, 0.46,
        '是'],
       ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.774, 0.376, '是'],
       ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.634, 0.264, '是'],
       ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', 0.608, 0.318, '是'],
       ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', 0.556, 0.215, '是'],
       ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.40299999999999997, 0.237,
        '是'],
       ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', 0.43700000000000006,
        0.21100000000000002, '是'],
       ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', 0.243, 0.267, '否'],
       ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', 0.36, 0.37, '否']],
      dtype=object)

### 连续数据

​ 选择合适的划分点，将数据进行二分（最简单）

​ 最优划分点的选取：

​（1）给定数据集D和连续特征A，假设A上有n个不同取值,从小到大排序得{a1,a2,…,an}。

​（2）令 t=(ai+ai+1)/2 ,则可以得到划分 <=ai以及 >=ai+1 的数据集

（3）生成n-1个候选集

（4）根据信息增益最大选出最优划分点

In [16]:
#连续数据的分离
def splitContinuousDataSet(dataSet,axis,value,direction=0):
    """
    采用二分法对连续变量进行划分,
    :param dataSet: 数据集 array
    :param axis:
    :param value:根据value划分为两部分
    :param direction:0-1取值，0表示<value 的数据； 1表示>=value的数据
    :return:
    """
    if direction==0:
        numberset = np.where(dataSet[:, axis] < value)
    else:
        numberset = np.where(dataSet[:, axis] >= value)
    return dataSet[numberset,][0]

In [17]:
def bestSplitforCondata(dataSet,axis):
    sorteddata = sorted(dataSet[:, axis]) #对该特征取值进行排序
    #生成候选集
    splitPoint = [(sorteddata[i] + sorteddata[i + 1]) / 2 for i in range(len(sorteddata) - 1)]
    LEN = len(dataSet[:,axis])
    bestsplitInfoGain = float('-inf')
    bestsplitevalue = float('-inf')
    Ent = calShannonEnt(dataSet)
    for value in splitPoint:
        conEnt = 0.0
        for i in range(2):
            spdata = splitContinuousDataSet(dataSet, axis, value, direction=i)
            Len = len(spdata[:,axis])
            conEnt += float(Len/LEN) * calShannonEnt(spdata)
        InfoEnt = Ent - conEnt
        if InfoEnt > bestsplitInfoGain:
            bestsplitInfoGain = InfoEnt
            bestsplitevalue = value
    return bestsplitInfoGain,bestsplitevalue

In [18]:
splitContinuousDataSet(data,6,0.7,direction=1)
print(bestSplitforCondata(data,6))

bestSplitforCondata(data,7)


(0.2624392604045631, 0.38149999999999995)


(0.34929372233065203, 0.126)

### 选择最优划分特征

In [19]:
def chooseBestFeaturetoSplit(dataSet,Method = 'ID3'):
    """
    需要考虑连续型变量和离散性变量
    :param dataSet:
    :param Method:
    :return:
    """
    numFeatures = np.shape(dataSet)[1] - 1
    bestEnt = calShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = None
    bestsplitevalue = None
    for i in range(numFeatures):
        if isinstance(dataSet[0,i],float): #判断是否是连续型变量
            infoGain,splitevalue = bestSplitforCondata(dataSet,i)
        else:
            infoGain = bestEnt - calconEnt(dataSet, i)
            splitevalue = None
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
            bestsplitevalue = splitevalue
    return bestFeature,bestInfoGain,bestsplitevalue

In [20]:
chooseBestFeaturetoSplit(data)

(3, 0.3805918973682686, None)

### 每个节点根据投票法确定类

In [21]:
def majorityCnt( dataSet ):
    labelsCount = Counter(dataSet[:, -1])
    LEN = len(dataSet[:,-1])
    label = labelsCount.most_common(1)[0][0]
    prob = round(labelsCount[label]/LEN,3)
    return label,prob

In [22]:
majorityCnt( data )

('否', 0.529)

### 构造决策树类

In [23]:
class decisiontree:
    def __init__(self,label,prob,axis = None,value =None,*node):
        self.label = label #该节点的类的标记
        self.prob = prob #该节点为该类的概率
        self.axis = axis #该节点采用哪个特征继续划分
        self.value = value #离散：特征的取值   连续：0-1 

### 生成树

In [24]:
def createTree(dataSet,e=0.001,num=2):
    """
    :param dataSet: 数据集 array
    :param e: 阈值，信息增益小于该阈值则停止
    :param num: 阈值，数据集中数目小于该数目则停止
    :return:返回树
    """
    label, prob = majorityCnt(dataSet)
    number = len(dataSet)
    tree = decisiontree(label, prob)
    axis, Gain,splitevalue = chooseBestFeaturetoSplit(dataSet)
    tree.axis = axis
    node = []
    if Gain < e:
        return tree
    elif number <= num:
        return tree
    else:
        if splitevalue == None:  #判断是否是连续型变量
            for value in set(dataSet[:, axis]):
                newset = splitDataSet(dataSet, axis, value)
                newtree = createTree(newset)
                newtree.value = value
                newtree.splitevalue = splitevalue
                node.append(newtree)
            tree.node = node
        else:
            for i in range(2):
                newset = splitContinuousDataSet(dataSet, axis, splitevalue, direction=i)
                newtree = createTree(newset)
                newtree.value = i
                newtree.splitevalue = splitevalue
                node.append(newtree)
            tree.node = node
    return tree

###  搜索决策树

In [25]:
def FinLabel(tree,sample):
    if not hasattr(tree,'node'):
        label,prob = tree.label , tree.prob
        return label,prob
    LEN = len(tree.node)
    axis = tree.axis
    num = -1
    for i in range(LEN):
        if tree.node[i].splitevalue == None:
            if sample[axis] == tree.node[i].value:
                num = i
            break
        else:
            if sample[axis] < tree.node[i].splitevalue and tree.node[i].value == 0:
                num = i
            if sample[axis] >= tree.node[i].splitevalue and tree.node[i].value == 1:
                num = i
            break
    label, prob = FinLabel(tree.node[num],sample)
    return label,prob

In [26]:
mytree = createTree(data)

In [45]:
feature

['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '密度', '含糖量', '好瓜']

In [27]:
vars(mytree)

{'axis': 3,
 'label': '否',
 'node': [<__main__.decisiontree at 0x1ddd2cc7198>,
 'prob': 0.529,
 'value': None}

In [41]:
print(vars(mytree.node[0]))
print(vars(mytree.node[1]))
print(vars(mytree.node[2]))

{'label': '否', 'prob': 1.0, 'axis': None, 'value': '模糊', 'splitevalue': None}
{'label': '是', 'prob': 0.778, 'axis': 6, 'value': '清晰', 'node': [<__main__.decisiontree object at 0x000001DDD2CCF978>, <__main__.decisiontree object at 0x000001DDD2CCF780>], 'splitevalue': None}
{'label': '否', 'prob': 0.8, 'axis': 5, 'value': '稍糊', 'node': [<__main__.decisiontree object at 0x000001DDD2CCF940>, <__main__.decisiontree object at 0x000001DDD2CCFA58>], 'splitevalue': None}


In [42]:
print(vars(mytree.node[1].node[0]))
print(vars(mytree.node[1].node[1]))
print(vars(mytree.node[2].node[0]))
print(vars(mytree.node[2].node[1]))

{'label': '否', 'prob': 1.0, 'axis': None, 'value': 0, 'splitevalue': 0.38149999999999995}
{'label': '是', 'prob': 1.0, 'axis': None, 'value': 1, 'splitevalue': 0.38149999999999995}
{'label': '是', 'prob': 1.0, 'axis': None, 'value': '软粘', 'splitevalue': None}
{'label': '否', 'prob': 1.0, 'axis': None, 'value': '硬滑', 'splitevalue': None}


In [28]:
sample = np.delete(data[0],-1,0)
FinLabel(mytree,sample)  #进行预测
('是', 1.0)

('是', 1.0)